# <b>2024 CSU Systemwide Business Analytics Competition
CSUS Center for Business Analytics
“Should This Loan be Approved or Denied?”
Predictive Modeling Using the SBA National Data

<b> Submission by: Joshua Battise, Celine Nguyen, Kiet Tran

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install DMBA mord

In [ ]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
from mord import LogisticIT
import matplotlib.pylab as plt
import seaborn as sns
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score
import math
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sys
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
print(sys.version)
print('The scikit-learn version is {}.'.format(sklearn.__version__))

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
The scikit-learn version is 1.5.2.


STORY BEHIND THE DATA: " This data set is from the U.S. Small Business Administration
(SBA) and provides historical data from 1987 through 2014, containing 27 variables and
899,164 observations.  Each observation represents a loan that was guaranteed to some degree
by the SBA. Included is a variable [MIS_Status] which indicates if the loan was paid in full or
defaulted/charged off"(Li, Mickel, Taylor (2018))

This project asks us to analyze the provided SBA loan data. Based on the data, we must analyze on what basis a loan may be Paid in Full (PIF) or Charged off (CHGOFF). Both of these options are categories under the column MIS_Status. This will be accomplished using numerous machine learning models. By doing this, we asses the Probability of Default (PD) for each loan application.

We must consider the balance between potential profit and the potenial loss of potential lending. This is why predicting loan outcomes is so important. Through this analysis, we may aid banks in avoiding granting loans likely to default and approving loans likely to be PIF.

1. Data Exploration and Preprocessing
How is the outcome variable MIS_Status distributed? Identify predictors that may help predict
MIS_Status using descriptive statistics and visualization.


<b>Column Descriptions as Provided:

Predictor           | Data Type    | Description
--------------------|-------------|----------------------------------------------------------
LoanNr_ChkDgt       | Text         | Identifier – Primary Key
Name                | Text         | Borrower Name
City                | Text         | Borrower City
State               | Text         | Borrower State
Zip                 | Text         | Borrower Zip Code
Bank                | Text         | Bank Name
BankState           | Text         | Bank State
NAICS               | Text         | North American Industry Classification System code
ApprovalDate        | Date/Time    | Date SBA Commitment Issued
ApprovalFY          | Text         | Fiscal Year of Commitment
Term                | Number       | Loan term in months
NoEmp               | Number       | Number of Business Employees
NewExist            | Text         | 1 = Existing Business, 2 = New Business
CreateJob           | Number       | Number of jobs created
RetainedJob         | Number       | Number of jobs retained
FranchiseCode       | Text         | Franchise Code 00000 or 00001 = No Franchise
UrbanRural          | Text         | 1 = Urban, 2 = Rural, 0 = Undefined
RevLineCr           | Text         | Revolving Line of Credit: Y = Yes
LowDoc              | Text         | LowDoc Loan Program: Y = Yes, N = No
ChgOffDate          | Date/Time    | The date when a loan is declared to be in default
DisbursementDate    | Date/Time    | Disbursement Date
DisbursementGross   | Currency     | Amount Disbursed
BalanceGross        | Currency     | Gross amount outstanding
MIS_Status          | Text         | Loan Status, “CHGOFF” (defaulted) or “P I F” (paid in full)
ChgOffPrinGr        | Currency     | Charged-off Amount
GrAppv              | Currency     | Gross Amount of Loan Approved by Bank
SBA_Appv            | Currency     | SBA’s Guaranteed Amount of Approved Loan


In [ ]:
DATA = Path('/content/drive/My Drive/dmba-datasets/dmba/')

In [ ]:
SBA_df = pd.read_csv(DATA / 'SBAnationalCompetition.csv')
SBA_df.head()

<ipython-input-140-03e29df5a5ac>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  SBA_df = pd.read_csv(DATA / 'SBAnationalCompetition.csv')


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


Shape of the dataframe:

In [ ]:
SBA_df.shape

(899164, 27)

# <B> Y Feature: MIS_Status

Determining our Y Feature before any data cleaning and selection is necessary as it establishes the objective and context of our analysis. This first step allows us to prioritize certain aspects of our data within the logical context of real-world applications. With this understanding, we can assume that a categorical input will be necessary for representing outcomes.

From the table above we see that MIS status has two options that must be converted into dummies. This is also necessary as we must create a 1/0 binary outcome for the PIF and CHGOFF. This binary representation of the "success class" is crucial for machine learning and modeling. With this code, 1 will represent loans that are CHGOFF and 0 represents the loans that were PIF.

In [ ]:
MIS_dummies = pd.get_dummies(SBA_df['MIS_Status'], prefix='Default', dtype=int)
SBA_df = pd.concat([SBA_df, MIS_dummies], axis=1)
SBA_df.drop('Default_P I F', axis=1, inplace=True)
SBA_df.drop('MIS_Status', axis=1, inplace=True)

In [ ]:
list(SBA_df.columns)

['LoanNr_ChkDgt',
 'Name',
 'City',
 'State',
 'Zip',
 'Bank',
 'BankState',
 'NAICS',
 'ApprovalDate',
 'ApprovalFY',
 'Term',
 'NoEmp',
 'NewExist',
 'CreateJob',
 'RetainedJob',
 'FranchiseCode',
 'UrbanRural',
 'RevLineCr',
 'LowDoc',
 'ChgOffDate',
 'DisbursementDate',
 'DisbursementGross',
 'BalanceGross',
 'ChgOffPrinGr',
 'GrAppv',
 'SBA_Appv',
 'Default_CHGOFF']

# Data Types and Data Cleaning

Identifying the data type of each column as well as the number of unique samples per column is important. This provides preliminary analysis which helps guide the selection of important vs. non-important features.

In [ ]:
list_item = []
for col in SBA_df.columns:
    list_item.append([col,  # Feature name
                      SBA_df[col].dtype,  # Data type
                      SBA_df[col].nunique()])  # Number of unique samples

dfDesc = pd.DataFrame(columns=['feature', 'data_type', '# of Unique Samples'], data=list_item)
dfDesc

,feature,data_type,# of Unique Samples
0,LoanNr_ChkDgt,int64,899164
1,Name,object,779583
2,City,object,32581
3,State,object,51
4,Zip,int64,33611
5,Bank,object,5802
6,BankState,object,56
7,NAICS,int64,1312
8,ApprovalDate,object,9859
9,ApprovalFY,object,70


Now, let us address the data being interpreted as the incorrect data type.

From this table, we can begin to address the columns being considered an object when they contain numerical data. This is an issue with many of the currency columns.

In [ ]:
for col in ['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']:
    SBA_df[col] = SBA_df[col].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)

Next I addressed Approval and Disbursement date and fixed them to date/time format.

In [ ]:
SBA_df['ApprovalDate'] = pd.to_datetime(SBA_df['ApprovalDate'], format='%d-%b-%y')
SBA_df['DisbursementDate'] = pd.to_datetime(SBA_df['DisbursementDate'], format='%d-%b-%y')

To figure out why Approval Fiscal Year was being identified as an object I called for an array of the unique values. 1976 A contains a string that must be cleaned to fix this issue.

In [ ]:
SBA_df['ApprovalFY'].unique()

array([1997, 1980, 2006, 1998, 1999, 2000, 2001, 1972, 2003, 2004, 1978,
       1979, 1981, 2005, 1962, 1982, 1965, 1966, 1983, 1973, 1984, 2007,
       1985, 1986, 1987, 2008, 1988, 2009, 1989, 1991, 1990, 1974, 2010,
       1967, 2011, 1992, 1993, 2002, 2012, 2013, 1994, 2014, 1975, 1977,
       1976, 1968, '2004', '1994', '1979', '1976', '1967', '1975', '1974',
       '1977', '1981', '1982', '1983', '1984', '1978', '1980', '1968',
       '1976A', '1969', '1995', '1970', '2005', '1996', '1971', 1996,
       1971], dtype=object)

In [ ]:
cleaned_approval_fy = []

for value in SBA_df['ApprovalFY']:
    if isinstance(value, str):
        cleaned_approval_fy.append(int(value.replace('A', '')))
    else:
        cleaned_approval_fy.append(value)

SBA_df['ApprovalFY'] = cleaned_approval_fy

The zip code must be changed to String as it is a classification, not a numerical value. The same is true for the Urban-Rural column.

In [ ]:
SBA_df = SBA_df.astype({'Zip': 'object', 'UrbanRural': 'object'})

### TRAIN TEST SPLIT

Before moving forward, we can talk about the importance of running train_test_split at this specific point in the notebook, as a means of protecting the original data frame, as well as protecting our validation set. As we move closer to feature selection and model building, this is vital to prevent an overly optimistic evaluation of our model performances.

In [ ]:
X = SBA_df.drop('Default_CHGOFF', axis=1)  # X Features
y = SBA_df['Default_CHGOFF']  # Y
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

Next, this code identifys the number of missing values in each column. This is an important step of preporcessing as missing values can negativley impact the quality of our analysis and modeling. For this reasons we will remove the missing values from our data.


In [ ]:
total_missing_train = train_X.isna().sum().sum()  # Sum across all columns in train_X
total_missing_valid = valid_X.isna().sum().sum()  # Sum across all columns in valid_X

total_missing = total_missing_train + total_missing_valid  # Total missing in both datasets

print("Total Missing Values (All Features):", total_missing)

Total Missing Values (All Features): 749262


In [ ]:
train_X.shape

(539498, 26)

In [ ]:
valid_X.shape

(359666, 26)

In [ ]:
train_y.shape

(539498,)

In [ ]:
valid_y.shape

(359666,)

When we made the table demonstrating unique values, the Franchise Code had 2768 despite its purpose of simply classifying whether or not the applicant was a franchise or not. To remedy this it must be sorted into franchise and non-franchise. The key for the predictor explains that 0-1 indicates a non-franchise. This means any value key above one should indicate a loan applicant who is a franchise. This code indirectly creates a Binary dummy variable hence the new column is named "Franchise_Dummy". This will make any data with a franchise code classified correctly rather than creating 2768 non-sense dummies which would ruin our analysis.

In [ ]:
train_X['Franchise_Dummy'] = train_X['FranchiseCode'].apply(lambda x: 1 if x > 1 else 0)
train_X.drop('FranchiseCode', axis=1, inplace=True)

valid_X['Franchise_Dummy'] = valid_X['FranchiseCode'].apply(lambda x: 1 if x > 1 else 0)
valid_X.drop('FranchiseCode', axis=1, inplace=True)

Although there weren't as many variations for RevLineCr and LowDoc, their purpose is classifcation meaning the data needs to be filtered down to a comprehensive yes or no in the context of its effect on the data. RevLineCr needs to be a 1 for data with Revolving Lines of Credit and 0 for all else. Similarly, LowDoc needs to be a 1 for loan applicants who get the Low Doc loan program and a 0 for all else.

New Exist is a classification of either "1" or "2" meaning it shouldn't be read as a float.

Let us observe the unique values outside of their binary classification

In [ ]:
SBA_df['RevLineCr'].unique()

array(['N', '0', 'Y', 'T', nan, '`', ',', '1', 'C', '3', '2', 'R', '7',
       'A', '5', '.', '4', '-', 'Q'], dtype=object)

In [ ]:
SBA_df['LowDoc'].unique()

array(['Y', 'N', 'C', '1', nan, 'S', 'R', 'A', '0'], dtype=object)

In [ ]:
SBA_df['NewExist'].unique()

array([ 2.,  1.,  0., nan])

In [ ]:
columns_to_process = ['RevLineCr', 'LowDoc', 'NewExist']

for column in columns_to_process:
    if column in ['NewExist', 'RevLineCr', 'LowDoc']:
        train_X[column] = train_X[column].apply(lambda x: 'New' if x == 2 else ('Existing' if x == 1 else 'Other') if column == 'NewExist' else (x if x in ['Y', 'N'] else 'Other'))
        valid_X[column] = valid_X[column].apply(lambda x: 'New' if x == 2 else ('Existing' if x == 1 else 'Other') if column == 'NewExist' else (x if x in ['Y', 'N'] else 'Other'))

    dummies_train = pd.get_dummies(train_X[column], prefix=column, dtype=int)
    dummies_valid = pd.get_dummies(valid_X[column], prefix=column, dtype=int)

    train_X = pd.concat([train_X, dummies_train], axis=1)
    valid_X = pd.concat([valid_X, dummies_valid], axis=1)

    train_X.drop([column], axis=1, inplace=True)
    valid_X.drop([column], axis=1, inplace=True)

In [ ]:
dtype_comparison = pd.concat([train_X.dtypes, valid_X.dtypes], axis=1)
dtype_comparison.columns = ['train_X', 'valid_X']
print(dtype_comparison)

                          train_X         valid_X
LoanNr_ChkDgt               int64           int64
Name                       object          object
City                       object          object
State                      object          object
Zip                        object          object
Bank                       object          object
BankState                  object          object
NAICS                       int64           int64
ApprovalDate       datetime64[ns]  datetime64[ns]
ApprovalFY                  int64           int64
Term                        int64           int64
NoEmp                       int64           int64
CreateJob                   int64           int64
RetainedJob                 int64           int64
UrbanRural                 object          object
ChgOffDate                 object          object
DisbursementDate   datetime64[ns]  datetime64[ns]
DisbursementGross         float64         float64
BalanceGross              float64         float64


NAICS Code Analysis

The NAICS data provided contains up to 6-digit codes with hyper-specific classifications of industry. Upon research, the table below demonstrates a broader categorization of the industry code by interpreting the first two numbers of the NAICS code provided.

Industry classification table provided from the [United States Bureau](https://www.census.gov/naics/?58967?yearbck=2022)

| Sector   | Definition                                                                          |
|----------|------------------------------------------------------------------------------------|
| 11       | Agriculture, Forestry, Fishing and Hunting                                        |
| 21       | Mining, Quarrying, and Oil and Gas Extraction                                     |
| 22       | Utilities                                                                         |
| 23       | Construction                                                                      |
| 31-33    | Manufacturing                                                                     |
| 42       | Wholesale Trade                                                                   |
| 44-45    | Retail Trade                                                                      |
| 48-49    | Transportation and Warehousing                                                   |
| 51       | Information                                                                       |
| 52       | Finance and Insurance                                                             |
| 53       | Real Estate and Rental and Leasing                                               |
| 54       | Professional, Scientific, and Technical Services                                  |
| 55       | Management of Companies and Enterprises                                          |
| 56       | Administrative and Support and Waste Management and Remediation Services         |
| 61       | Educational Services                                                             |
| 62       | Health Care and Social Assistance                                                |
| 71       | Arts, Entertainment, and Recreation                                              |
| 72       | Accommodation and Food Services                                                  |
| 81       | Other Services (except Public Administration)                                    |
| 92       | Public Administration                                                            |


This code will first create a new column (IndustryCode) in the data frame for us to store our industry classifications. By applying the lambda function, the major sector codes are extracted from our existing data and categorized based on their first two digits.

In [ ]:
industry_mapping = {
    '11': 'Agriculture',
    '21': 'Mining',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing', '32': 'Manufacturing', '33': 'Manufacturing',
    '42': 'Wholesale',
    '44': 'Retail', '45': 'Retail',
    '48': 'Transportation', '49': 'Transportation',
    '51': 'Information',
    '52': 'Finance',
    '53': 'Real_Estate',
    '54': 'Professional_Services',
    '55': 'Management',
    '56': 'Admin_Waste',
    '61': 'Education',
    '62': 'Healthcare',
    '71': 'Arts_Recreation',
    '72': 'Food_Services',
    '81': 'Other_Services',
    '92': 'Public_Admin'
}

In [ ]:
train_X['IndustryCode'] = train_X['NAICS'].astype(str).str[:2].apply(
    lambda x: industry_mapping.get(x, 'Other')
)
train_X.drop(['NAICS'], axis=1, inplace=True)

valid_X['IndustryCode'] = valid_X['NAICS'].astype(str).str[:2].apply(
    lambda x: industry_mapping.get(x, 'Other')
)
valid_X.drop(['NAICS'], axis=1, inplace=True)

# X Feature Analysis and Selection:

| Predictor         | Reason for Exclusion                                                                   |
|-------------------|-----------------------------------------------------------------------------------------|
| LoanNr_ChkDgt     | Identifier key with no inherent value for modeling                                     |
| Name              | Identifier with no inherent value for modeling                                         |
| City              | Highly specific with many unique values, difficult to process, unlikely to affect predictions|
| Zipcode           | Highly specific with many unique values, difficult to process, unlikely to affect predictions|
| Bank              | Identifier with no inherent value                                                     |
| SBA_Appv          | Guaranteed amount is based on percentage of gross loan, not dollar amount              |                           |
| ApprovalDate and DisbursementDate  | Date values arerent necssary or relevant due to the inclusion of ApprovalFY  which are highly correlated. (See Matrix Below)     |
| ChgOffPrinGr and ChgOffDate | We are trying to predict the likelihood of default. These predictors are only known after a loan has defaulted.

Upon reviewing the variables, we hypothesized that Approval Date, Disbursement Date, and ApprovalFy would have a high correlation with each other. If they do, it would be beneficial to the simplicity of the model to drop the two datetime-formated features. We will create and interpret a correlation matrix below.

In [ ]:
train_X['ApprovalDate_Num'] = (train_X['ApprovalDate'].to_numpy().astype('datetime64[D]') - np.datetime64('1970-01-01')) / np.timedelta64(1, 'D')
train_X['DisbursementDate_Num'] = (train_X['DisbursementDate'].to_numpy().astype('datetime64[D]') - np.datetime64('1970-01-01')) / np.timedelta64(1, 'D')

correlation_matrix_train = train_X[['ApprovalDate_Num', 'DisbursementDate_Num', 'ApprovalFY']].corr()
print("Correlation Matrix (Training Set):")
print(correlation_matrix_train)

# Drop the numeric date variables
train_X.drop(['ApprovalDate_Num', 'DisbursementDate_Num'], axis=1, inplace=True)

Correlation Matrix (Training Set):
                      ApprovalDate_Num  DisbursementDate_Num  ApprovalFY
ApprovalDate_Num              1.000000              0.996007    0.998027
DisbursementDate_Num          0.996007              1.000000    0.995010
ApprovalFY                    0.998027              0.995010    1.000000


As we assumed, these time-based features have a very strong correlation. It is important to drop the predictors that could be overfitting the model through redundancy.

We assumed that Bank State and State (borrower) would have a high correlation. We did some outside research to find the correlation between these nominal variables. The organization [GeeksforGeeks](https://www.geeksforgeeks.org/how-to-calculate-cramers-v-in-python) demonstrates how to find the "Cramer's V Value" in Python which could demonstrate the correlation of these variables. After reading the linked article, we applied the methods to our utilized context. GeeksforGeeks utlizes the library scipy stats so we imported that below.

In [ ]:
import scipy.stats as stats

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    rcorr = r - ((r - 1)**2) / (n - 1)
    kcorr = k - ((k - 1)**2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

# Calculate Cramer's V for BankState and State using training data
cramers_v_value = cramers_v(train_X['BankState'], train_X['State'])

print(f"Cramer's V between BankState and State (Training Data): {cramers_v_value}")

# Create a correlation matrix (single value in this case)
correlation_matrix = pd.DataFrame(
    data=[[cramers_v_value]],
    index=['BankState'],
    columns=['State']
)

print("\nCorrelation Matrix (Cramer's V - Training Data):")
print(correlation_matrix)

Cramer's V between BankState and State (Training Data): 0.6398363679216205

Correlation Matrix (Cramer's V - Training Data):
              State
BankState  0.639836


The Cramer's V value of 0.639836 shows we were correct with our hypothesis.

Common interpretations of Cramer's V:

0.0 - 0.2: Weak association
0.2 - 0.6: Moderate association
0.6 - 1.0: Strong association

 The value indicates a strong correlation between the state of the borrower and the bank. This is great as we can highly reduce the dimensionality of our data by dropping bank state  from the model. Keeping both BankState and State would be redundant and potentially overfit. This correlation likely stems from real world context of Banks lending to borrowers in shared reigions.

This code will drop ALL the features we explained above

In [ ]:
columns_to_drop = ['LoanNr_ChkDgt','BankState', 'Name', 'City', 'Zip', 'Bank', 'SBA_Appv','ApprovalDate','DisbursementDate','ChgOffDate','ChgOffPrinGr']
train_X = train_X.drop(columns=columns_to_drop, axis=1)
valid_X = valid_X.drop(columns=columns_to_drop, axis=1)

In [ ]:
list(train_X.columns)

['State',
 'ApprovalFY',
 'Term',
 'NoEmp',
 'CreateJob',
 'RetainedJob',
 'UrbanRural',
 'DisbursementGross',
 'BalanceGross',
 'GrAppv',
 'Franchise_Dummy',
 'RevLineCr_N',
 'RevLineCr_Other',
 'RevLineCr_Y',
 'LowDoc_N',
 'LowDoc_Other',
 'LowDoc_Y',
 'NewExist_Existing',
 'NewExist_New',
 'NewExist_Other',
 'IndustryCode']

With so many dummy variables created for the states and industries, we will be checking the relationship between defaulted loans and these features. Features within the same category with similar relationships to the likelihood of a default, will be combined into one feature. This will aid in reducing the chances of overfitting with our future models.

In [ ]:
train_X_with_target = pd.concat([train_X, train_y], axis=1)

for col in ['State', 'IndustryCode']:
    pivot_table = pd.pivot_table(train_X_with_target, values='Default_CHGOFF', index=col, aggfunc=np.mean)
    print(f"Pivot Table for {col}:\n{pivot_table}\n")

train_X = train_X_with_target.drop(columns=['Default_CHGOFF'])

<ipython-input-168-c12c8564e3e8>:4: FutureWarning: The provided callable <function mean at 0x7c8d36b8d5a0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pivot_table = pd.pivot_table(train_X_with_target, values='Default_CHGOFF', index=col, aggfunc=np.mean)
<ipython-input-168-c12c8564e3e8>:4: FutureWarning: The provided callable <function mean at 0x7c8d36b8d5a0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pivot_table = pd.pivot_table(train_X_with_target, values='Default_CHGOFF', index=col, aggfunc=np.mean)


Pivot Table for State:
       Default_CHGOFF
State                
AK           0.110198
AL           0.165833
AR           0.166886
AZ           0.206760
CA           0.184721
CO           0.180494
CT           0.138186
DC           0.246632
DE           0.185406
FL           0.275115
GA           0.245582
HI           0.135508
IA           0.118306
ID           0.138845
IL           0.226170
IN           0.175597
KS           0.133733
KY           0.184222
LA           0.179651
MA           0.125016
MD           0.197348
ME           0.105889
MI           0.222846
MN           0.116259
MO           0.156576
MS           0.159275
MT           0.066869
NC           0.191339
ND           0.082069
NE           0.115908
NH           0.107163
NJ           0.203942
NM           0.112015
NV           0.234764
NY           0.198977
OH           0.165512
OK           0.157948
OR           0.146576
PA           0.144365
RI           0.115885
SC           0.201127
SD           0.080089
TN       

The features with similar relationships to default are listed below with their new combined predictor name.

| New Feature Name | Combined Features |
|---|---|
| State_GA_NV | State_GA, State_NV |
| State_IL_MI | State_IL, State_MI |
| State_NJ_SC_NY_MD | State_NJ, State_SC, State_NY, State_MD |
| State_TX_CA_DE_KY_VA | State_TX, State_CA, State_DE, State_KY, State_VA |
| State_CO_IN_MA | State_CO, State_IN, State_MA |
| State_UT_WY_ND_SD | State_UT, State_WY, State_ND, State_SD |
| Industry_Finan_Trans | Finance, Transportation |
| Industry_Edu_Info_Admin | Education, Information, Admin_Waste |
| Industry_Const_Retail | Construction, Retail |
| Industry_Food_Arts | Food_Services, Arts_Recreation |
| Industry_Other_Whole_Prof | Other_Services, Wholesale, Professional_Services |
| Industry_Util_Manu_Pub | Utilities, Manufacturing, Public_Admin |
| Industry_Health_Mgmt_Mining_Agri | Healthcare, Management, Mining, Agriculture |

In [ ]:
#State
train_X['State_GA_NV'] = [1 if d in ("State_GA", "State_NV",) else 0 for d in train_X.State]
train_X['State_IL_MI'] = [1 if d in ("State_IL", "State_MI") else 0 for d in train_X.State]
train_X['State_NJ_SC_NY_MD'] = [1 if d in ("State_NJ", "State_SC", "State_NY", "State_MD") else 0 for d in train_X.State]
train_X['State_TX_CA_DE_KY_VA'] = [1 if d in ("State_TX", "State_CA", "State_DE", "State_KY", "State_VA") else 0 for d in train_X.State]
train_X['State_CO_IN_MA'] = [1 if d in ("State_CO", "State_IN", "State_MA") else 0 for d in train_X.State]
train_X['State_UT_WY_ND_SD'] = [1 if d in ("State_UT", "State_WY", "State_ND", "State_SD") else 0 for d in train_X.State]
#Industry
train_X['Industry_Finan_Trans'] = [1 if d in ("Finance", "Transportation",) else 0 for d in train_X.IndustryCode]
train_X['Industry_Edu_Info_Admin'] = [1 if d in ("Education", "Information", "Admin_Waste") else 0 for d in train_X.IndustryCode]
train_X['Industry_Const_Retail'] = [1 if d in ("Construction", "Retail") else 0 for d in train_X.IndustryCode]
train_X['Industry_Food_Arts'] = [1 if d in ("Food_Services", "Arts_Recreation") else 0 for d in train_X.IndustryCode]
train_X['Industry_Other_Whole_Prof'] = [1 if d in ("Other_Services", "Wholesale", "Professional_Services") else 0 for d in train_X.IndustryCode]
train_X['Industry_Util_Manu_Pub'] = [1 if d in ("Utilities", "Manufacturing", "Public_Admin") else 0 for d in train_X.IndustryCode]
train_X['Industry_Health_Mgmt_Mining_Agri'] = [1 if d in ("Healthcare", "Management", "Mining", "Agriculture") else 0 for d in train_X.IndustryCode]

In [ ]:
#State
valid_X['State_GA_NV'] = [1 if d in ("State_GA", "State_NV",) else 0 for d in valid_X.State]
valid_X['State_IL_MI'] = [1 if d in ("State_IL", "State_MI") else 0 for d in valid_X.State]
valid_X['State_NJ_SC_NY_MD'] = [1 if d in ("State_NJ", "State_SC", "State_NY", "State_MD") else 0 for d in valid_X.State]
valid_X['State_TX_CA_DE_KY_VA'] = [1 if d in ("State_TX", "State_CA", "State_DE", "State_KY", "State_VA") else 0 for d in valid_X.State]
valid_X['State_CO_IN_MA'] = [1 if d in ("State_CO", "State_IN", "State_MA") else 0 for d in valid_X.State]
valid_X['State_UT_WY_ND_SD'] = [1 if d in ("State_UT", "State_WY", "State_ND", "State_SD") else 0 for d in valid_X.State]
#Industry
valid_X['Industry_Finan_Trans'] = [1 if d in ("Finance", "Transportation",) else 0 for d in valid_X.IndustryCode]
valid_X['Industry_Edu_Info_Admin'] = [1 if d in ("Education", "Information", "Admin_Waste") else 0 for d in valid_X.IndustryCode]
valid_X['Industry_Const_Retail'] = [1 if d in ("Construction", "Retail") else 0 for d in valid_X.IndustryCode]
valid_X['Industry_Food_Arts'] = [1 if d in ("Food_Services", "Arts_Recreation") else 0 for d in valid_X.IndustryCode]
valid_X['Industry_Other_Whole_Prof'] = [1 if d in ("Other_Services", "Wholesale", "Professional_Services") else 0 for d in valid_X.IndustryCode]
valid_X['Industry_Util_Manu_Pub'] = [1 if d in ("Utilities", "Manufacturing", "Public_Admin") else 0 for d in valid_X.IndustryCode]
valid_X['Industry_Health_Mgmt_Mining_Agri'] = [1 if d in ("Healthcare", "Management", "Mining", "Agriculture") else 0 for d in valid_X.IndustryCode]

For the features that did not share a close relationship, a binary dummy must still be added. However, whe useing pd.getdummies, binary dummies will also be made for the features we combined above. For this reason we will need to run an extensive drop code to ensure to repeat binary dummies of features that have already been combined.

In [ ]:
categorical_features = ['State', 'IndustryCode', 'UrbanRural']

for feature in categorical_features:
    # Create dummy variables for training data
    encoded_features_train = pd.get_dummies(train_X[feature], prefix=feature, dtype=int)
    train_X = pd.concat([train_X, encoded_features_train], axis=1)
    train_X.drop(feature, axis=1, inplace=True)

    # Create dummy variables for validation data
    encoded_features_valid = pd.get_dummies(valid_X[feature], prefix=feature, dtype=int)
    valid_X = pd.concat([valid_X, encoded_features_valid], axis=1)
    valid_X.drop(feature, axis=1, inplace=True)

In [ ]:
train_X.drop(columns=[
    # Original State columns
    "State_GA", "State_NV", "State_IL", "State_MI",
    "State_NJ", "State_SC", "State_NY", "State_MD",
    "State_TX", "State_CA", "State_DE", "State_KY", "State_VA",
    "State_CO", "State_IN", "State_MA",
    "State_UT", "State_WY", "State_ND", "State_SD",
    # Original Industry columns with "IndustryCode_" prefix
    "IndustryCode_Finance", "IndustryCode_Transportation",
    "IndustryCode_Education", "IndustryCode_Information", "IndustryCode_Admin_Waste",
    "IndustryCode_Construction", "IndustryCode_Retail",
    "IndustryCode_Food_Services", "IndustryCode_Arts_Recreation",
    "IndustryCode_Other_Services", "IndustryCode_Wholesale", "IndustryCode_Professional_Services",
    "IndustryCode_Utilities", "IndustryCode_Manufacturing", "IndustryCode_Public_Admin",
    "IndustryCode_Healthcare", "IndustryCode_Management", "IndustryCode_Mining", "IndustryCode_Agriculture"
], inplace=True)

valid_X.drop(columns=[
    # Original State columns
    "State_GA", "State_NV", "State_IL", "State_MI",
    "State_NJ", "State_SC", "State_NY", "State_MD",
    "State_TX", "State_CA", "State_DE", "State_KY", "State_VA",
    "State_CO", "State_IN", "State_MA",
    "State_UT", "State_WY", "State_ND", "State_SD",
    # Original Industry columns with "IndustryCode_" prefix
    "IndustryCode_Finance", "IndustryCode_Transportation",
    "IndustryCode_Education", "IndustryCode_Information", "IndustryCode_Admin_Waste",
    "IndustryCode_Construction", "IndustryCode_Retail",
    "IndustryCode_Food_Services", "IndustryCode_Arts_Recreation",
    "IndustryCode_Other_Services", "IndustryCode_Wholesale", "IndustryCode_Professional_Services",
    "IndustryCode_Utilities", "IndustryCode_Manufacturing", "IndustryCode_Public_Admin",
    "IndustryCode_Healthcare", "IndustryCode_Management", "IndustryCode_Mining", "IndustryCode_Agriculture"
], inplace=True)

In [ ]:
list(train_X.columns)

['ApprovalFY',
 'Term',
 'NoEmp',
 'CreateJob',
 'RetainedJob',
 'DisbursementGross',
 'BalanceGross',
 'GrAppv',
 'Franchise_Dummy',
 'RevLineCr_N',
 'RevLineCr_Other',
 'RevLineCr_Y',
 'LowDoc_N',
 'LowDoc_Other',
 'LowDoc_Y',
 'NewExist_Existing',
 'NewExist_New',
 'NewExist_Other',
 'State_GA_NV',
 'State_IL_MI',
 'State_NJ_SC_NY_MD',
 'State_TX_CA_DE_KY_VA',
 'State_CO_IN_MA',
 'State_UT_WY_ND_SD',
 'Industry_Finan_Trans',
 'Industry_Edu_Info_Admin',
 'Industry_Const_Retail',
 'Industry_Food_Arts',
 'Industry_Other_Whole_Prof',
 'Industry_Util_Manu_Pub',
 'Industry_Health_Mgmt_Mining_Agri',
 'State_AK',
 'State_AL',
 'State_AR',
 'State_AZ',
 'State_CT',
 'State_DC',
 'State_FL',
 'State_HI',
 'State_IA',
 'State_ID',
 'State_KS',
 'State_LA',
 'State_ME',
 'State_MN',
 'State_MO',
 'State_MS',
 'State_MT',
 'State_NC',
 'State_NE',
 'State_NH',
 'State_NM',
 'State_OH',
 'State_OK',
 'State_OR',
 'State_PA',
 'State_RI',
 'State_TN',
 'State_VT',
 'State_WA',
 'State_WI',
 'State

In [ ]:
print("Value Counts for Training Data (train_y):")
print(train_y.value_counts())

print("\nValue Counts for Validation Data (valid_y):")
print(valid_y.value_counts())

print("\n---------------------\n")

total_count_0 = train_y.value_counts()[0] + valid_y.value_counts()[0]  # Total '0's
total_count_1 = train_y.value_counts()[1] + valid_y.value_counts()[1]  # Total '1's

grand_total = total_count_0 + total_count_1

print("Grand Total (All Loans):", grand_total)
print("Original Dataframe:", SBA_df.shape)

Value Counts for Training Data (train_y):
Default_CHGOFF
0    444904
1     94594
Name: count, dtype: int64

Value Counts for Validation Data (valid_y):
Default_CHGOFF
0    296702
1     62964
Name: count, dtype: int64

---------------------

Grand Total (All Loans): 899164
Original Dataframe: (899164, 27)


In [ ]:
SBA_df.shape

(899164, 27)

We avoided data loss by cleaning the data and categorical features. As we see in the shape of the dataset, there should be 899,164 rows of data. Between the training and validation set, we have this meaning no data has been dropped.

In [ ]:
total_missing_train = train_X.isna().sum().sum()  # Sum across all columns in train_X
total_missing_valid = valid_X.isna().sum().sum()  # Sum across all columns in valid_X

total_missing = total_missing_train + total_missing_valid  # Total missing in both datasets

print("Total Missing Values (All Features):", total_missing)

Total Missing Values (All Features): 0


This information shows that many of the features no longer contain large amounts of missing values. However, knowing how missing data from future potential data should be handled is good practice. Modeling future data by simply dropping the rows with missing data highly alters the distribution of the data. This could be not representative of the true patterns or connections. For this reason, one should use imputation. However simple imputation of just the mean/mode from the data in a data set this large can also poorly affect the datas intergirty. Upon further research, implementing [Sci-Kit Learn's KNNImputer](https://https://scikit-learn.org/1.5/modules/generated/sklearn.impute.KNNImputer.html) would be our choice. The KNNImputer will fill in the Missing values of the data set by finding the nearest neighbors of each data point with missing values. This would highly impove our analysis quality than the alternative option of either dropping data or imputing less accurate points.

In [ ]:
train_X.shape

(539498, 67)

In [ ]:
train_y.shape

(539498,)

##Conclusion

In [ ]:
from google.colab import files  # Import files for download

# ... (Previous code for data processing) ...

# Define file names for download
train_X_file_name = 'train_X.csv'
train_y_file_name = 'train_y.csv'
valid_X_file_name = 'valid_X.csv'
valid_y_file_name = 'valid_y.csv'

# Save dataframes to temporary CSV files
train_X.to_csv(train_X_file_name, index=False)
train_y.to_csv(train_y_file_name, index=False)
valid_X.to_csv(valid_X_file_name, index=False)
valid_y.to_csv(valid_y_file_name, index=False)

# Download files using google.colab.files.download
files.download(train_X_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download(train_y_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download(valid_X_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download(valid_y_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>